In [3]:
# Author: Hangting Chen
# Copyright: Tencent AI Lab
# Paper: Ultra Dual-Path Compression For Joint Echo Cancellation and Noise Suppression
# This code give the source of the ultra dual-path compression module and an example of usage.

In [4]:
import torch
import torch.nn as nn
import numpy as np
import math

# Define neural network modules

In [5]:
class WAVSTFT(nn.Module):
    def __init__(self, win_size=320):
        super(WAVSTFT,self).__init__()
        window = torch.from_numpy(np.hanning(win_size).astype(np.float32))
        self.window_size = window.shape[-1]
        self.hop_length =  self.window_size // 2
        window = window.unsqueeze(0).unsqueeze(-1)
        divisor = torch.ones(1,1,1,self.window_size*4)
        divisor = nn.functional.unfold(divisor,(1,self.window_size),stride=self.hop_length)
        divisor = divisor * window.pow(2.0)
        divisor = nn.functional.fold(divisor,(1,self.window_size*4),(1,self.window_size),stride=self.hop_length)[:,0,0,:]
        divisor = divisor[0,self.window_size:2*self.window_size].unsqueeze(0).unsqueeze(-1)
        self.register_buffer('window', window)
        self.register_buffer('divisor', divisor)

    def magphase(self, complex_tensor: torch.Tensor):
        mag = complex_tensor.pow(2.).sum(-1).pow(0.5 * 1.0)
        phase = torch.atan2(complex_tensor[..., 1], complex_tensor[..., 0])
        return mag, phase

    def add_window(self, x, divisor):
        out = x * self.window / divisor
        return out

    def frame(self,x):
        assert x.dim()==2, x.shape
        out = x.unsqueeze(1).unsqueeze(1)
        out = nn.functional.pad(out, (self.window_size, self.window_size), 'constant', 0)
        out = nn.functional.unfold(out,(1,self.window_size),\
            stride=self.hop_length) # B N T
        return out

    def overlap_and_add(self,x,length):
        assert x.dim()==3, x.shape
        out = nn.functional.fold(x,(1,length+2*self.window_size),(1,self.window_size), \
            stride=self.hop_length)[:,0,0,:]
        out = out[:,self.window_size:-self.window_size]
        return out

    def rfft(self, x):
        assert x.dim()==3, x.shape
        return torch.fft.rfft(x, dim=1)

    def irfft(self, x):
        assert x.dim()==3, x.shape
        return torch.fft.irfft(x, dim=1)

    def STFT(self, x, return_aux=False):
        assert x.dim()==2, x.shape
        out = self.frame(x)
        out = self.add_window(out, 1)
        out = self.rfft(out)
        if(return_aux):
            mag, phase = self.magphase(torch.view_as_real(out))
            lps = torch.log(mag**2 + 1e-8)
            return out, mag, phase, lps 
        else:
            return out

    def iSTFT(self, x, length):
        assert x.dim()==3, x.shape
        out = self.irfft(x)
        out = self.add_window(out, self.divisor)
        out = self.overlap_and_add(out, length=length)
        return out

In [6]:
class TimeCompression(nn.Module):
    def __init__(self, dim1, dim2, dim3, dim4, steps):
        super().__init__()
        self.dim1 = dim1 # input dim
        self.dim2 = dim2 # hidden dim
        self.dim3 = dim3 # input dim for decompression
        self.dim4 = dim4 # out dim
        self.steps = steps
        self.trans1 = nn.Conv2d(self.dim1 * self.steps, self.dim2, 1, bias = False)
        self.trans2 = nn.Conv2d(self.dim3, self.dim4, 1, bias = False)

    def forward(self, x, inverse):
        # x B C T F
        if(inverse):
            B, C, T, F = x.shape
            x = self.trans2(x).reshape(B, -1, 1, T, F).permute(0,1,3,2,4).contiguous() # B C T S F
            x = x.repeat(1,1,1,self.steps,1).reshape(B, -1, T*self.steps, F) # B C T*S F
            x = torch.nn.functional.pad(x,(0,0,self.steps-1,0),'constant',0)
            x = x[:,:,:-self.steps+1,:]
            if(self.pad > 0): x = x[:,:,self.pad:,:]
            return x
        else:
            B, C, T, F = x.shape
            if(x.shape[-2]%self.steps==0):
                self.pad = 0
            else:
                self.pad = self.steps - x.shape[-2]%self.steps
                x = torch.nn.functional.pad(x,(0,0,self.pad,0),'constant',0)
            x = x.reshape(B, C, -1, self.steps, F).permute(0,1,3,2,4).contiguous() # B C S T F
            x = x.reshape(B, C*self.steps, -1, F) # B C*S T F
            return self.trans1(x)

In [7]:
class FreqCompression(nn.Module):
    def __init__(self, nfreq, nfilters, in_dim, hidden_dim, \
        out_dim, sample_rate=16000):
        super().__init__()
        self.nfreq = nfreq
        self.nfilters = nfilters
        self.sample_rate = sample_rate
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim

        mel_scale = 'htk'
        
        all_freqs = torch.linspace(0, sample_rate // 2, nfreq)
        # calculate mel freq bins
        m_min = self._hz_to_mel(0, mel_scale=mel_scale)
        m_max = self._hz_to_mel(sample_rate/2.0, mel_scale=mel_scale)

        m_pts = torch.linspace(m_min, m_max, self.nfilters + 2)
        f_pts = self._mel_to_hz(m_pts, mel_scale=mel_scale)
        self.bounds = [0,]
        for freq_inx in range(1, len(f_pts)-1):
            self.bounds.append((all_freqs > f_pts[freq_inx]).float().argmax().item())
        self.bounds.append(nfreq)
        self.trans1 = nn.ModuleList()
        self.trans2 = nn.ModuleList()
        for freq_inx in range(self.nfilters):
            self.trans1.append(nn.Linear((self.bounds[freq_inx+2]-self.bounds[freq_inx])*self.in_dim, self.hidden_dim, bias=False))
            self.trans2.append(nn.Conv1d(self.hidden_dim, (self.bounds[freq_inx+2]-self.bounds[freq_inx])*self.out_dim, 1))
            
    def _hz_to_mel(self, freq: float, mel_scale: str = "htk") -> float:
        r"""
        Source: https://pytorch.org/audio/stable/_modules/torchaudio/functional/functional.html
        Convert Hz to Mels.

        Args:
            freqs (float): Frequencies in Hz
            mel_scale (str, optional): Scale to use: ``htk`` or ``slaney``. (Default: ``htk``)

        Returns:
            mels (float): Frequency in Mels
        """

        if mel_scale not in ["slaney", "htk"]:
            raise ValueError('mel_scale should be one of "htk" or "slaney".')

        if mel_scale == "htk":
            return 2595.0 * math.log10(1.0 + (freq / 700.0))

        # Fill in the linear part
        f_min = 0.0
        f_sp = 200.0 / 3

        mels = (freq - f_min) / f_sp

        # Fill in the log-scale part
        min_log_hz = 1000.0
        min_log_mel = (min_log_hz - f_min) / f_sp
        logstep = math.log(6.4) / 27.0

        if freq >= min_log_hz:
            mels = min_log_mel + math.log(freq / min_log_hz) / logstep

        return mels
    
    def _mel_to_hz(self, mels: torch.Tensor, mel_scale: str = "htk") -> torch.Tensor:
        """
        Source: https://pytorch.org/audio/stable/_modules/torchaudio/functional/functional.html
        Convert mel bin numbers to frequencies.

        Args:
            mels (torch.Tensor): Mel frequencies
            mel_scale (str, optional): Scale to use: ``htk`` or ``slaney``. (Default: ``htk``)

        Returns:
            freqs (torch.Tensor): Mels converted in Hz
        """

        if mel_scale not in ["slaney", "htk"]:
            raise ValueError('mel_scale should be one of "htk" or "slaney".')

        if mel_scale == "htk":
            return 700.0 * (10.0 ** (mels / 2595.0) - 1.0)

        # Fill in the linear scale
        f_min = 0.0
        f_sp = 200.0 / 3
        freqs = f_min + f_sp * mels

        # And now the nonlinear scale
        min_log_hz = 1000.0
        min_log_mel = (min_log_hz - f_min) / f_sp
        logstep = math.log(6.4) / 27.0

        log_t = mels >= min_log_mel
        freqs[log_t] = min_log_hz * torch.exp(logstep * (mels[log_t] - min_log_mel))

        return freqs
    
    def forward(self, x, inverse):
        if(inverse):
            # B C T F
            out = torch.zeros([x.shape[0],self.out_dim,self.nfreq,x.shape[2]], dtype=x.dtype, layout=x.layout, device=x.device)
            for freq_inx in range(self.nfilters):
                out[:,:,self.bounds[freq_inx]:self.bounds[freq_inx+2],:] = out[:,:,self.bounds[freq_inx]:self.bounds[freq_inx+2],:] + \
                    self.trans2[freq_inx](x[:,:,:,freq_inx]).reshape(x.shape[0],self.out_dim,-1,x.shape[-2])
            out[:,:,self.bounds[1]:self.bounds[-2],:] = out[:,:,self.bounds[1]:self.bounds[-2],:] / 2.0
            out = out.permute(0,1,3,2).contiguous().tanh()
            return out
        else:
            x = x.reshape(x.shape[0],self.in_dim, *x.shape[-2:]) # B C T F
            x = x.permute(0,2,1,3).contiguous() # B T C F
            x = torch.stack([self.trans1[freq_inx](x[:,:,:,self.bounds[freq_inx]:self.bounds[freq_inx+2]].flatten(start_dim=2)) \
                for freq_inx in range(self.nfilters)],-1) # B T C F
            x = x.permute(0,2,1,3).contiguous()
            return x

In [8]:
class UltraDualPath(nn.Module):
    def __init__(self, nfreq, in_dim, hidden_dim, out_dim, \
            freq_cprs_ratio, time_cprs_ratio):
        super(UltraDualPath, self).__init__()
        self.nfreq = nfreq
        self.in_dim = in_dim
        self.hidden_dim = hidden_dim
        self.out_dim = out_dim
        self.freq_cprs_ratio = freq_cprs_ratio
        self.time_cprs_ratio = time_cprs_ratio
        self.compress_modules = [
            TimeCompression(self.in_dim, self.in_dim * 2, \
                self.out_dim, self.out_dim, time_cprs_ratio), \
            FreqCompression(self.nfreq, self.nfreq // self.freq_cprs_ratio, \
                self.in_dim * 2, self.hidden_dim, self.out_dim), \
        ]
    def forward(self, x, inverse):
        out = x
        # print(out.shape)
        if(inverse):
            for m in self.compress_modules[::-1]:
                out = m(out, inverse)
                # print(out.shape)
        else:
            for m in self.compress_modules:
                out = m(out, inverse)
                # print(out.shape)
        return out

In [30]:
class DynamicLayerNorm(nn.Module):
    def __init__(self):
        super(DynamicLayerNorm, self).__init__()     

    def forward(self, x):
        return nn.functional.layer_norm(x,[x.shape[-1],])
    
class GRUBlock(nn.Module):
    def __init__(self, hidden_size, causal):
        super(GRUBlock, self).__init__()
        self.gru = nn.GRU(hidden_size, hidden_size, 1, bidirectional=False if causal else True, batch_first=True)
        self.linear = nn.Linear(hidden_size if causal else hidden_size*2, hidden_size)
        self.norm = DynamicLayerNorm()
        self.activation = nn.ReLU()

    def forward(self, input):
        out, _ = self.gru(input) # T B E
        out = self.linear(self.activation(out))
        out = input + out
        out = self.norm(out)
        return out

class ExampleDualPathBackbone(nn.Module):
    # An example of dual path module
    def __init__(self, hidden_size, num_layers, causal=True):
        super(ExampleDualPathBackbone, self).__init__()
        self.hidden_size = hidden_size

        # dual-path RNN
        self.row_trans = nn.ModuleList([])
        self.col_trans = nn.ModuleList([])
        for i in range(num_layers):
            self.row_trans.append(
                GRUBlock(hidden_size, False)
            )
            self.col_trans.append(
                GRUBlock(hidden_size, causal)
            )

    def forward(self, input):
        #  input --- [b,  c,  num_frames, frame_size]  --- [b, c, dim2, dim1]
        b, _, dim2, dim1 = input.shape
        output = input
        for i in range(len(self.row_trans)):
            row_input = output.permute(0, 2, 3, 1).contiguous().view(b*dim2, dim1, -1)  # [b*dim2, dim1, c]
            output = self.row_trans[i](row_input)  # [b*dim2, dim1, c]
            output = output.view(b, dim2, dim1, -1).permute(0, 3, 1, 2).contiguous()  # [b, c, dim2, dim1]

            col_input = output.permute(0, 3, 2, 1).contiguous().view(b*dim1, dim2, -1)  # [b*dim1, dim2, c]
            output = self.col_trans[i](col_input)  # [b*dim1, dim2, c]
            output = output.view(b, dim1, dim2, -1).permute(0, 3, 2, 1).contiguous()  # [b, c, dim2, dim1]

        return output

In [89]:
class ExampleModel(nn.Module):
    def __init__(self):
        super(ExampleModel, self).__init__()  
        win_size = 320 # 20ms for 16khz
        hidden_size = 48
        freq_cprs_ratio = 4
        time_cprs_ratio = 4
        num_in_channels = 3
        num_dual_path_blocks = 4
        hidden_size_post = 4

        self.wavSTFT = WAVSTFT(win_size) # 16khz, 20ms/10ms -> 320 samples
        self.ultraCompress = UltraDualPath(win_size//2 + 1, num_in_channels*2, hidden_size, hidden_size, freq_cprs_ratio, time_cprs_ratio) # 3ch*(real+imag)
        self.dualpath = ExampleDualPathBackbone(hidden_size, num_dual_path_blocks, True)
        self.conv2mask = nn.Conv2d(hidden_size, num_in_channels*2, 1, 1)

        self.postCompress = FreqCompression(win_size // 2 + 1, (win_size // 2 + 1) // 2, num_in_channels + 1, 1, hidden_size_post)
        self.postNet = nn.GRU((win_size // 2 + 1) // 2, (win_size // 2 + 1) // 2, 1, bidirectional=False, batch_first=True)
        self.postLinear2mask = nn.Sequential(
            nn.Linear(win_size // 2 + 1, win_size // 2 + 1),
            nn.Conv2d(hidden_size_post, hidden_size_post, 1),
            nn.Sigmoid(),
            nn.Linear(win_size // 2 + 1, win_size // 2 + 1),
            nn.Conv2d(hidden_size_post, 1, 1),
            nn.Sigmoid(),
        )

    def complex2logmag(self, real, imag):
        return (real.pow(2.0)+imag.pow(2.0)+1e-8).log()

    def forward(self, wav):
        # organize input
        B, C, N = wav.shape
        spec = self.wavSTFT.STFT(wav.reshape(B*C, N))
        spec = torch.view_as_real(spec.reshape(B, C, *spec.shape[-2:])) # B C F T 2
        spec = spec.permute(0,1,4,3,2) # B C 2 T F
        spec = spec.reshape(B, C*2, *spec.shape[-2:])

        # compress
        latent = self.ultraCompress(spec,0)
        # run dual-path network
        latent = self.dualpath(latent)
        # decompress
        latent = self.ultraCompress(latent,1)
        # conv2mask
        mask = self.conv2mask(latent).reshape(B, C, 2, *spec.shape[-2:])
        spec = spec.reshape(B, C, 2, *spec.shape[-2:])
        est_spec = torch.stack([
            spec[:,:,0] * mask[:,:,0] - spec[:,:,1] * mask[:,:,1], \
            spec[:,:,0] * mask[:,:,1] + spec[:,:,1] * mask[:,:,0], \
        ],2) # B C 2 T F
        est_spec = est_spec.sum(1, keepdim=True) # B 1 2 T F

        # organize input for postnet
        post_input = torch.cat([spec, est_spec], 1)
        post_input = self.complex2logmag(post_input[:,:,0], post_input[:,:,1]) # B C+1 T F
        # compress
        latent = self.postCompress(post_input, 0)
        # run postnet
        latent, _ = self.postNet(latent.squeeze(1))
        # decompress
        latent = self.postCompress(latent.unsqueeze(1), 1) # B 4 T F
        # conv2mask
        latent = self.postLinear2mask(latent) # B 1 T F
        est_spec = est_spec + spec[:,[0]] * latent.unsqueeze(2) 
        ## we assume the first channel includes full information (e.g., the mic signal of AEC, the ref channel in multi-channel microphone)
        ## thus it is used to complement spec loss from the previous stage

        # recover to wav
        est_spec = torch.complex(est_spec.sum(1)[:,0], est_spec.sum(1)[:,1])
        ests_wav = self.wavSTFT.iSTFT(est_spec.permute(0,2,1), N)
        ests_wav = ests_wav.reshape(B, 1, N)

        return ests_wav

# An example of run

## Define model and input

In [66]:
## Network configuration
win_size = 320
hidden_size = 48
freq_cprs_ratio = 4
time_cprs_ratio = 4
num_in_channels = 3

## Define network modules
wavSTFT = WAVSTFT(win_size) # 16khz, 20ms/10ms -> 320 samples
ultraCompress = UltraDualPath(win_size//2 + 1, num_in_channels*2, hidden_size, hidden_size, freq_cprs_ratio, time_cprs_ratio) # 3ch*(real+imag)
exampleDualPathModule = ExampleDualPathBackbone(hidden_size, 2, True)

## Define input
wav = torch.rand(2,num_in_channels,16000) # nbatch=2, nchannels=3, nsamples=16000

## Run dual path compression

In [67]:
# organize input
B, C, N = wav.shape
spec = wavSTFT.STFT(wav.reshape(B*C, N))
spec = torch.view_as_real(spec.reshape(B, C, *spec.shape[-2:])) # B C F T 2
spec = spec.permute(0,1,4,3,2) # B C 2 T F
spec = spec.reshape(B, C*2, *spec.shape[-2:])
print(spec.shape) # nbatch, nchannels*2, nframes, nfreqs

torch.Size([2, 6, 103, 161])


In [68]:
# compress
latent = ultraCompress(spec,0)
print(latent.shape) # nbatch, hidden_dim, nframes, nfreqs

torch.Size([2, 48, 26, 40])


In [69]:
# run dual-path network
latent = exampleDualPathModule(latent)
print(latent.shape) # nbatch, hidden_dim, nframes, nfreqs

torch.Size([2, 48, 26, 40])


In [70]:
# decompress
output = ultraCompress(latent,1)
print(output.shape) # nbatch, out_dim, nframes, nfreqs

torch.Size([2, 48, 103, 161])


## Run freq compression only

In [71]:
# Frequcy only
ultraCompress = FreqCompression(win_size // 2 + 1, (win_size // 2 + 1) // freq_cprs_ratio, num_in_channels * 2, hidden_size, hidden_size) # 4x compression ratio
# compress
latent = ultraCompress(spec, 0)
print(latent.shape)  # nbatch, hidden_dim, nframes, nfreqs
# run dual path network
latent = exampleDualPathModule(latent)
print(latent.shape)  # nbatch, hidden_dim, nframes, nfreqs
# decompress
output = ultraCompress(latent, 1)
print(output.shape)  # nbatch, out_dim, nframes, nfreqs

torch.Size([2, 48, 103, 40])
torch.Size([2, 48, 103, 40])
torch.Size([2, 48, 103, 161])


## Run time compression only

In [72]:
# Frequcy only
ultraCompress = TimeCompression(num_in_channels * 2, hidden_size, hidden_size, hidden_size, time_cprs_ratio) # 4x compression ratio
# compress
latent = ultraCompress(spec, 0)
print(latent.shape)  # nbatch, hidden_dim, nframes, nfreqs
# run dual path network
latent = exampleDualPathModule(latent)
print(latent.shape)  # nbatch, hidden_dim, nframes, nfreqs
# decompress
output = ultraCompress(latent, 1)
print(output.shape)  # nbatch, out_dim, nframes, nfreqs

torch.Size([2, 48, 26, 161])
torch.Size([2, 48, 26, 161])
torch.Size([2, 48, 103, 161])


# Run ExampleModel

In [90]:
## Define input
wav = torch.rand(2,num_in_channels,16000) # nbatch=2, nchannels=3, nsamples=16000

## Define model
model = ExampleModel()
est_wav = model(wav)
print(est_wav.shape)

torch.Size([2, 1, 16000])


In [92]:
from thop import profile, clever_format
net_causal = ExampleModel()
macs, params, ret_dict = profile(net_causal.to(torch.device('cpu')), inputs=(torch.rand(1,3,25120), ), ret_layer_info=True,)
macs, params = clever_format([macs, params], "%.3f")
from pprint import pprint
pprint(ret_dict)
print('Params : {}, Macs : {:.03f}'.format(params, float(macs[:-1])/1.6))

[INFO] Register count_gru() for <class 'torch.nn.modules.rnn.GRU'>.
[INFO] Register count_linear() for <class 'torch.nn.modules.linear.Linear'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.activation.ReLU'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv2d'>.
[INFO] Register count_convNd() for <class 'torch.nn.modules.conv.Conv1d'>.
[INFO] Register zero_ops() for <class 'torch.nn.modules.container.Sequential'>.
{'conv2mask': (7418880.0, 294.0, {}),
 'dualpath': (321638400.0,
              197376.0,
              {'col_trans': (107212800.0,
                             65856.0,
                             {'0': (26803200.0,
                                    16464.0,
                                    {'activation': (0.0, 0.0, {}),
                                     'gru': (23116800.0, 14112.0, {}),
                                     'linear': (3686400.0, 2352.0, {}),
                                     'norm': (0.0, 0, {})}),
                 